# EX11 Solutions

This notebook generates synthetic data for Exercise 11 when executed. The generated files are ignored by git via `.gitignore`.


In [1]:
import os
import glob
import numpy as np
from PIL import Image

DATA_DIR = 'EX11_synthetic'
os.makedirs(DATA_DIR, exist_ok=True)

poses = []
for idx in range(1, 6):
    img = np.zeros((64, 64, 3), dtype=np.uint8)
    img[..., 0] = idx * 40
    img[..., 1] = np.arange(64).reshape(64, 1)
    img[..., 2] = np.arange(64).reshape(1, 64)
    Image.fromarray(img).save(os.path.join(DATA_DIR, f'{idx:06d}.png'))
    pose = np.eye(4)
    pose[0, 3] = idx * 0.1
    poses.append(pose)
poses = np.stack(poses)
np.save(os.path.join(DATA_DIR, 'poses.npy'), poses)

print('Generated', len(glob.glob(os.path.join(DATA_DIR, '*.png'))), 'images')
print('Poses shape:', poses.shape)


Generated 5 images
Poses shape: (5, 4, 4)
